<a href="https://colab.research.google.com/github/zhangxs131/small_tools_for_nlp/blob/main/gender_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#根据名字判别性别

这里使用作者数据集，源项目地址为 

https://github.com/observerss/ngender

但作者没有给出源数据集，给出的数据集为单个汉字在男女名字中出现次数，所以本来想使用1-2gram进行改进也不行了。

这里还将总人数直接按照名字出现male，female次数相加得到，也不准确，总体还凑合吧。

In [6]:
!wget https://raw.githubusercontent.com/observerss/ngender/master/ngender/charfreq.csv

--2022-04-20 11:58:51--  https://raw.githubusercontent.com/observerss/ngender/master/ngender/charfreq.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90935 (89K) [text/plain]
Saving to: ‘charfreq.csv’

charfreq.csv        100%[===================>]  88.80K  --.-KB/s    in 0.01s   

2022-04-20 11:58:51 (6.47 MB/s) - ‘charfreq.csv’ saved [90935/90935]



In [7]:
import pandas as pd

df=pd.read_csv('charfreq.csv')
df.head(5)

,char,male,female
0,明,378860,63221
1,伟,378757,51232
2,军,378096,29518
3,建,366515,51477
4,华,344928,174529


In [12]:
sum={'male':df['male'].sum(),'female':df['female'].sum()}
sum

{'female': 10787021, 'male': 21280545}

In [66]:
text='小明'

for i in text:
  print(df['female'][df['char']==i].iloc[0])

120069
63221


In [87]:
#这里使用朴素贝叶斯计算概率
def prob_for_gender(firstname, gender='male'):
  t=(sum['male']+sum['female']) 
  p = 1. * sum[gender]/t  

  for char in firstname:
    a=df[gender][df['char']==char].iloc[0]
    p *= a/sum[gender]

  return p

def guess(name):
  firstname = name[1:]
  for char in firstname:
    assert u'\u4e00' <= char <= u'\u9fa0', u'姓名必须为中文'
  
  pf = prob_for_gender(firstname,'female')
  pm = prob_for_gender(firstname,'male')


  if pm > pf:
    return ('male', 1. * pm / (pm + pf))
  elif pm < pf:
    return ('female', 1. * pf / (pm + pf))
  else:
    return ('unknown', 0)

In [88]:
guess('张晓松')

('male', 0.8404324443331157)

In [89]:
guess('刘琦')

('male', 0.5768939671378696)

In [90]:
guess('张小明')

('male', 0.776631528500599)

In [91]:
guess("张宛若")

('female', 0.8456031689879948)

In [92]:
guess('赵本山')

('male', 0.9836229687547046)

In [93]:
guess('宋丹丹')

('female', 0.9759486128949907)

In [94]:
guess('范冰冰')

('female', 0.6241854276885191)

In [95]:
guess('杨一')

('male', 0.7450759960428096)

In [96]:
guess('石少丽')

('female', 0.8636462271424227)